# Código para aplicação das LB direto no dado para rotulação

## Preparando o test da base ouro para aplicar as labels functions

In [298]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [299]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [300]:
#apenas para testar o iob com spacy
#cria um novo dataset com as 2000 primeiras linhas
#csv_reader = pd.read_csv("/home/lucelia_vieira/Experimentos/ner/CSVs/REL_EXTRATO_CONTRATO_COMPLETO.csv", nrows=200)

In [301]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [302]:
#Carrega o dataset de teste para aplicacao das LB
#data = pd.read_csv('./CSVs/V2/split/Extrato_Contrato_Split_4_x_test_texto.csv', dtype=str)
data = pd.read_csv('./CSVs/V2/split/Suspensao_Licitação_Split_1_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/split/Aditamento_Contrato_Split_4_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/split/Anul_Revog_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/split/Aviso_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/split/Extrato_Convenio_Split_4_x_test_texto.csv', dtype=str)


data

,Unnamed: 0,Texto
0,0,AVISO DE SUSPENSÃO DE LICITAÇÃO PREGÃO ELETRÔN...
1,1,AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO SRP Nº...
2,2,AVISO DE SUSPENSAO PREGAO ELETRONICO No 481/20...
3,3,AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO Nº 70/...
4,4,AVISO DE SUSPENSÃO Comunicamos aos interessado...
5,5,AVISO DE SUSPENSÃO DE LICITAÇÃO LIC SERVIÇOS C...
6,6,AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO Nº 04/202...
7,7,AVISO DE SUSPENSAO PREGAO ELETRONICO No 24/201...
8,8,AVISO DE SUSPENSAO O Instituto do Cancer Infan...
9,9,AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO POR SRP N...


In [303]:
#criando novo dataframe somente com os textos
df1 = data['Texto'].reset_index(drop=True)
#df1 = data['texto'].reset_index(drop=True)
df1

0     AVISO DE SUSPENSÃO DE LICITAÇÃO PREGÃO ELETRÔN...
1     AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO SRP Nº...
2     AVISO DE SUSPENSAO PREGAO ELETRONICO No 481/20...
3     AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO Nº 70/...
4     AVISO DE SUSPENSÃO Comunicamos aos interessado...
5     AVISO DE SUSPENSÃO DE LICITAÇÃO LIC SERVIÇOS C...
6     AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO Nº 04/202...
7     AVISO DE SUSPENSAO PREGAO ELETRONICO No 24/201...
8     AVISO DE SUSPENSAO O Instituto do Cancer Infan...
9     AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO POR SRP N...
10    AVISO DE SUSPENSÃO E NOVA DATA DO PREGÃO ELETR...
11    AVISO DE SUSPENSAO O Instituto do Cancer Infan...
12    AVISO DE SUSPENSAO PREGAO ELETRONICO INTERNACI...
13    AVISO DE SUSPENSÃO " SINE DIE " PREGÃO ELETRÔN...
14    AVISO DE SUSPENSÃO SINE DIE CONCORRÊNCIA Nº 01...
15    AVISO DE SUSPENSÃODE LICITAÇÃO PREGÃO ELETRÔNI...
16    AVISO DE SUSPENSÃO REGIME DIFERENCIADO DE CONT...
Name: Texto, dtype: object

In [304]:
#len(df_sk)

In [305]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA
#Para rodar extrato de contrato descomente abaixo as duas linhas
#import dodfSkweak_Contrato
#skw = dodfSkweak_Contrato.SkweakContratos(df1)

#Para rodar aditamento de contrato descomente abaixo as duas linhas
#import dodfSkweak_Aditamento_2
#skw = dodfSkweak_Aditamento_2.SkweakAditamento(df1)

#Para rodar Aviso Licitação
#import dodfSkweak_Aviso_Licitacao
#skw = dodfSkweak_Aviso_Licitacao.SkweakAvisoLicitacao(df1)

#Para rodar Suspensao descomente abaixo as duas linhas
import dodfSkweak_Suspensao_1
skw = dodfSkweak_Suspensao_1.SkweakSuspensao(df1)

#Para rodar Anulacao e Revogação
#import dodfSkweak_Anul_Revog
#skw = dodfSkweak_Anul_Revog.SkweakAnulRevog(df1)

#Para rodar Convenio
#import dodfSkweak_Convenio_1
#skw = dodfSkweak_Convenio_1.SkweakConvenio(df1)

skw.apply_label_functions()
skw.train_HMM_Dodf()

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


Starting iteration 1
Finished E-step with 14 documents
Starting iteration 2


         1        -363.9500             +nan
         2        -348.1939         +15.7561
         3        -347.7491          +0.4448


Finished E-step with 14 documents
Starting iteration 3
Finished E-step with 14 documents
Starting iteration 4
Finished E-step with 14 documents


         4        -347.4032          +0.3459




doc.spans 
{'detec_processo': [00010.00001572/ 2021-10], 'processo_detector': [00010.00001572/ 2021-10], 'detec_modalidade': [PREGÃO ELETRÔNICO], 'detec_numero_licitacao': [37/2021], 'detec_processo_ml': [], 'detec_numero_licitacao_ml': [], 'hmm': [PREGÃO ELETRÔNICO, 00010.00001572/ 2021-10]}


doc.ents 
(PREGÃO ELETRÔNICO, 00010.00001572/ 2021-10)


doc.spans 
{'detec_processo': [00220-00005432/ 2021-82 (], 'processo_detector': [00220-00005432/ 2021-82], 'detec_modalidade': [PREGÃO ELETRÔNICO, CONCORRÊNCIA, PREGÃO ELETRÔNICO, Concorrência, PREGÃO ELETRÔNICO, Concorrência], 'detec_numero_licitacao': [001/2022, 001/2022, 001/2022], 'detec_processo_ml': [], 'detec_numero_licitacao_ml': [], 'hmm': [PREGÃO ELETRÔNICO, 00220-00005432/ 2021-82 (, PREGÃO ELETRÔNICO, PREGÃO ELETRÔNICO]}


doc.ents 
(PREGÃO ELETRÔNICO, 00220-00005432/ 2021-82 (, PREGÃO ELETRÔNICO, PREGÃO ELETRÔNICO)


doc.spans 
{'detec_processo': [], 'processo_detector': [], 'detec_modalidade': [PREGAO ELETRONICO], 'detec_nu

In [306]:
skw.list_spans_specific(0)

[PREGÃO ELETRÔNICO, 00010.00001572/ 2021-10]


In [307]:
#skw.list_spans_all()

In [308]:
data_label=skw.get_hmm_dataframe()
data_label

,processo_gdf,modalidade_licitacao,numero_licitacao,text,labels
0,00010.00001572/ 2021-10,PREGÃO ELETRÔNICO,,AVISO DE SUSPENSÃO DE LICITAÇÃO PREGÃO ELETRÔN...,O O O O O B-modalidade_licitacao I-modalidade_...
1,00220-00005432/ 2021-82 (,PREGÃO ELETRÔNICO,,AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO SRP Nº...,O O O O B-modalidade_licitacao I-modalidade_li...
2,,PREGAO ELETRONICO,,AVISO DE SUSPENSAO PREGAO ELETRONICO No 481/20...,O O O B-modalidade_licitacao I-modalidade_lici...
3,1.047/2021,Pregão Eletrônico,,AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO Nº 70/...,O O O O B-modalidade_licitacao I-modalidade_li...
4,112-00022751/ 2020-07,Pregão Eletrônico,,AVISO DE SUSPENSÃO Comunicamos aos interessado...,O O O O O O O B-modalidade_licitacao I-modalid...
5,,,,AVISO DE SUSPENSÃO DE LICITAÇÃO LIC SERVIÇOS C...,O O O O O O O O O O O O O O O O O O O O O O O ...
6,00055-00022429/ 2020-92,PREGÃO ELETRÔNICO,,AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO Nº 04/202...,O O O B-modalidade_licitacao I-modalidade_lici...
7,054.003.435/2017,PREGAO ELETRONICO,,AVISO DE SUSPENSAO PREGAO ELETRONICO No 24/201...,O O O B-modalidade_licitacao I-modalidade_lici...
8,,,,AVISO DE SUSPENSAO O Instituto do Cancer Infan...,O O O O O O O O O O O O O O O O O O O O O O O ...
9,00060-00169477/ 2017-41 ),PREGÃO ELETRÔNICO,,AVISO DE SUSPENSÃO PREGÃO ELETRÔNICO POR SRP N...,O O O B-modalidade_licitacao I-modalidade_lici...


In [309]:
# converting to CSV file
#data_label.to_csv("./CSVs/ANUL_REVOG_SUPERVISÃO_FRACA_TESTE.csv")

In [310]:
corpus = skw.get_IOB()

In [311]:
len(corpus)

17

In [312]:
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y)


In [313]:
debug

{'B-modalidade_licitacao',
 'B-processo_gdf',
 'I-modalidade_licitacao',
 'I-processo_gdf',
 'O'}

In [314]:
len(termos_skweak)

17

In [315]:
termos_skweak_list = []
for i in termos_skweak:
   if not isinstance(i, list):
      termos_skweak_list .append(i)
   else:
      for j in i:
       termos_skweak_list .append(j)

len(termos_skweak_list )

2244

In [316]:
#termos_skweak_list

In [317]:
#apenas para conferencia
#df = pd.DataFrame(termos_skweak_list, columns=['Texto']) 
#df.to_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/termos_skweak_list.csv')

In [318]:
labels_skweak_list = []
for i in labels_skweak:
   if not isinstance(i, list):
      labels_skweak_list.append(i)
   else:
      for j in i:
        labels_skweak_list.append(j)

len(labels_skweak_list)

2244

## Preparando o test da base ouro para fazer a validação

In [319]:
#Carrega o dataset de teste da base ouro para aplicar a validação
#df_gold_ato= pd.read_csv('./CSVs/Extrato_Contrato_Split_1_x_test.csv', dtype=str)
#df_gold_ato

In [320]:
#Carrega os atos do dataset de teste da base ouro
#https://pt.stackoverflow.com/questions/232208/ler-uma-lista-de-listas-em-txt-python-3
import pickle
#Contrato
#with open("./CSVs/V2/split/Extrato_Contrato_Split_4_x_test.txt", "rb") as file:
#Aditamento
#with open("./CSVs/V2/split/Aditamento_Contrato_Split_4_x_test.txt", "rb") as file:
#Suspensao
with open("./CSVs/V2/split/Suspensao_Licitação_Split_1_x_test.txt", "rb") as file:
#Aviso Licitacao
#with open("./CSVs/V2/split/Extrato_Contrato_Split_0_x_test.txt", "rb") as file: 
#Anul e Revog
#with open("./CSVs/V2/split/Suspensao_Licitação_Split_4_x_test.txt", "rb") as file: 
#Convenio
#with open("./CSVs/V2/split/Extrato_Convenio_Split_4_x_test.txt", "rb") as file:      
    x_test = pickle.load(file)

In [321]:
# Retira uma nível da lista dos atos do dataset de teste da base ouro
x_test_list = []
for i in x_test:
   if not isinstance(i, list):
      x_test_list.append(i)
   else:
      for j in i:
        x_test_list.append(j)

len(x_test_list)

2244

In [322]:
#Carrega os labels do dataset de teste da base ouro (Lista dentro de lista)
import pickle
#Contrato
#with open("./CSVs/V2/split/Extrato_Contrato_Split_4_y_test.txt", "rb") as file:
#Aditamento
#with open("./CSVs/V2/split/Aditamento_Contrato_Split_4_y_test.txt", "rb") as file:
#Suspensao
with open("./CSVs/V2/split/Suspensao_Licitação_Split_1_y_test.txt", "rb") as file:
#Aviso Licitacao
#with open("./CSVs/V2/split/Extrato_Contrato_Split_0_y_test.txt", "rb") as file: 
#Anul e Revog
#with open("./CSVs/V2/split/Suspensao_Licitação_Split_0_y_test.txt", "rb") as file:
#Convenio
#with open("./CSVs/V2/split/Extrato_Convenio_Split_4_y_test.txt", "rb") as file:        
    y_test = pickle.load(file)

In [323]:
#  Retira uma nível da lista dos labels do dataset de teste da base ouro

y_test_list = []
for i in y_test:
   if not isinstance(i, list):
      y_test_list.append(i)
   else:
      for j in i:
       y_test_list.append(j)

len(y_test_list)

2244

In [324]:
len(termos_skweak_list)

2244

In [325]:
#Apenas checa se os tokens estão batento
resultados = []
for sk_ato , gold_ato in zip(termos_skweak_list, x_test_list):
    tupla=(sk_ato , gold_ato) 
    resultados.append(tupla)
    if sk_ato != gold_ato:
        print(sk_ato, gold_ato)

#for item in resultados:
#    print(item)


In [326]:

''' classes = ['B-numero_contrato',
 'B-processo_gdf',
 'I-processo_gdf',
 'B-data_assinatura_contrato',
 'I-numero_contrato',
 'I-data_assinatura_contrato'] '''


" classes = ['B-numero_contrato',\n 'B-processo_gdf',\n 'I-processo_gdf',\n 'B-data_assinatura_contrato',\n 'I-numero_contrato',\n 'I-data_assinatura_contrato'] "

In [327]:
#len(labels_skweak_list)

In [328]:
#classes

In [329]:
#Validação com sklearn
#metrics.flat_f1_score(y_test, labels_skweak, average='weighted', labels=classes)

In [330]:
#Validação com seqeval
from seqeval.metrics import f1_score
f1_score(y_test, labels_skweak, average='weighted')

0.24924924924924927

In [331]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=False, mode='strict', scheme=IOB2)
print(report)


                      precision    recall  f1-score   support

                  50       0.00      0.00      0.00         1
        decisao_tcdf       0.00      0.00      0.00         5
modalidade_licitacao       0.94      0.75      0.83        20
    nome_responsavel       0.00      0.00      0.00        17
    numero_licitacao       0.00      0.00      0.00        20
    objeto_licitacao       0.00      0.00      0.00        14
     orgao_licitante       0.00      0.00      0.00        17
        processo_gdf       0.85      0.85      0.85        13

           micro avg       0.90      0.24      0.38       107
           macro avg       0.22      0.20      0.21       107
        weighted avg       0.28      0.24      0.26       107



/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [332]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=True, mode='strict', scheme=IOB2)
print(report)
np.save("./Results/V2/METRICA_SUSPENSAO_LICITACAO_LB_DIRETO_1.npy", report)

{'50': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'decisao_tcdf': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'modalidade_licitacao': {'precision': 0.9375, 'recall': 0.75, 'f1-score': 0.8333333333333334, 'support': 20}, 'nome_responsavel': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 17}, 'numero_licitacao': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 20}, 'objeto_licitacao': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 14}, 'orgao_licitante': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 17}, 'processo_gdf': {'precision': 0.8461538461538461, 'recall': 0.8461538461538461, 'f1-score': 0.8461538461538461, 'support': 13}, 'micro avg': {'precision': 0.896551724137931, 'recall': 0.24299065420560748, 'f1-score': 0.3823529411764706, 'support': 107}, 'macro avg': {'precision': 0.22295673076923078, 'recall': 0.19951923076923078, 'f1-score': 0.20993589743589744, 'support': 107}, 'weig

In [333]:
#classification_report sklearn
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
   gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=False)
print(report)

NameError: name 'gold_y_test' is not defined

In [ ]:
import numpy as np
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=True)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_LB_DIRETO_V1_1.npy", report)